In [ ]:
from fastdownload import download_url
from fastai.vision.all import *
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO
from pathlib import Path

# Download model if not already present
url = 'https://drive.google.com/uc?export=download&id=1YrxKIKF1koAfsm42jLvJxwTwLI2hoxkG'
if not Path('export2.pkl').exists():
    print("Downloading model (please wait)...")
    download_url(url, dest='export2.pkl', show_progress=False)  # Disable progress bar for Voila
    print("Download complete.")
else:
    print("Model already downloaded.")

# Load model
learn_inf = load_learner('export2.pkl')

# Upload and button widgets
btn_upload = widgets.FileUpload(accept='image/*')
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

# Classification function
def on_click_classify(change):
    if not btn_upload.value:
        lbl_pred.value = "⚠️ Please upload an image before classifying."
        return
    uploaded_file = next(iter(btn_upload.value.values()))
    img_bytes = uploaded_file['content']
    img = PILImage.create(BytesIO(img_bytes))
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(256, 256))
    try:
        pred, pred_idx, probs = learn_inf.predict(img)
        lbl_pred.value = f'✅ Prediction: {pred}; Probability: {probs[pred_idx]:.4f}'
    except Exception as e:
        lbl_pred.value = f"❌ Error during prediction: {e}"

# Connect button
btn_run.on_click(on_click_classify)

# Layout
ui = widgets.VBox([
    widgets.Label('⬇️ Upload your image here ⬇️'),
    btn_upload,
    btn_run,
    out_pl,
    lbl_pred
])

# Display the UI
display(ui)
